In [2]:
import cv2
from PIL import Image
import torch
from transformers import AutoImageProcessor, DetrForObjectDetection

2023-12-15 11:55:42.204908: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-15 11:55:42.270708: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-15 11:55:43.765494: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:


# Load the pre-trained model and image processor
image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")

# Open the camera
cap = cv2.VideoCapture(2)  # 0 represents the default camera

frame_skip = 5  # Process every 5th frame
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Skip frames
    if frame_skip > 0:
        frame_skip -= 1
        continue
    else:
        frame_skip = 5  # Reset frame_skip

    # Downsample the frame to reduce resolution
    frame = cv2.resize(frame, (640, 480))

    # Convert the OpenCV BGR image to RGB (PIL format)
    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Process the image with the object detection model
    inputs = image_processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    # Convert outputs to COCO API
    target_sizes = torch.tensor([image.size[::-1]])
    results = image_processor.post_process_object_detection(outputs, threshold=0.9, target_sizes=target_sizes)[0]

    # Draw bounding boxes on the frame
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
        box = [round(i, 2) for i in box.tolist()]

        # Convert relative coordinates to absolute coordinates
        h, w, _ = frame.shape
        box[0] *= w
        box[1] *= h
        box[2] *= w
        box[3] *= h

        cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
        label_str = f"{model.config.id2label[label.item()]}: {round(score.item(), 3)}"
        cv2.putText(frame, label_str, (int(box[0]), int(box[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Object Detection', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


In [ ]:
stream = cv2.VideoCapture(2)

In [3]:
from torch import hub # Hub contains other models like FasterRCNN
model = torch.hub.load( \
                      'ultralytics/yolov5', \
                      'yolov5s', \
                      pretrained=True)

/home/kakas/anaconda3/lib/python3.11/site-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /home/kakas/.cache/torch/hub/master.zip


  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/f4/79/b192c64e13eefc7f8543a10080767ab057850fa9bdbc38e7c261a3432c93/ultralytics-8.0.227-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 1.2 MB/s eta 0:00:0000:0100:01
requirements: Ultralytics requirement ['Pillow>=10.0.1'] not found, attempting AutoUpdate...


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tts 0.17.8 requires cython==0.29.30, which is not installed.
tts 0.17.8 requires einops==0.6.*, which is not installed.
tts 0.17.8 requires encodec==0.1.*, which is not installed.
tts 0.17.8 requires g2pkk>=0.1.1, which is not installed.
tts 0.17.8 requires k-diffusion, which is not installed.
tts 0.17.8 requires umap-learn==0.5.*, which is not installed.
tts 0.17.8 requires fsspec==2023.6.0, but you have fsspec 2023.4.0 which is incompatible.
tts 0.17.8 requires gruut[de,es,fr]==2.2.3, but you have gruut 2.3.4 which is incompatible.
tts 0.17.8 requires inflect==5.6.*, but you have inflect 7.0.0 which is incompatible.
tts 0.17.8 requires packaging==23.1, but you have packaging 23.0 which is incompatible.
tts 0.17.8 requires scipy>=1.11.2, but you have scipy 1.10.1 which is incompatible.
tts 0.17.8 requires tqdm==4

  Obtaining dependency information for Pillow>=10.0.1 from https://files.pythonhosted.org/packages/6f/d8/f31dd84b4363b5f24c71b25a13ec3855f5ff233e07e1c3f1f8e979e12be2/Pillow-10.1.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 1.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 13.3s, installed 1 package: ['Pillow>=10.0.1']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2023-12-15 Python-3.11.4 torch-2.1.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 48.0MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [4]:
def score_frame(frame, model):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)
    frame = [torch.tensor(frame)]
    results = self.model(frame)
    labels = results.xyxyn[0][:, -1].numpy()
    cord = results.xyxyn[0][:, :-1].numpy()
    return labels, cord

In [5]:
def plot_boxes(self, results, frame):
    labels, cord = results
    n = len(labels)
    x_shape, y_shape = frame.shape[1], frame.shape[0]
    for i in range(n):
        row = cord[i]
        # If score is less than 0.2 we avoid making a prediction.
        if row[4] < 0.2: 
            continue
        x1 = int(row[0]*x_shape)
        y1 = int(row[1]*y_shape)
        x2 = int(row[2]*x_shape)
        y2 = int(row[3]*y_shape)
        bgr = (0, 255, 0) # color of the box
        classes = self.model.names # Get the name of label index
        label_font = cv2.FONT_HERSHEY_SIMPLEX #Font for the label.
        cv2.rectangle(frame, \
                      (x1, y1), (x2, y2), \
                       bgr, 2) #Plot the boxes
        cv2.putText(frame,\
                    classes[labels[i]], \
                    (x1, y1), \
                    label_font, 0.9, bgr, 2) #Put a label over box.
        return frame

In [ ]:
from time import time
import numpy as np


In [ ]:
player = stream
assert player.isOpened()
x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
#four_cc = cv2.VideoWriter_fourcc(*"MJPG")
#out = cv2.VideoWriter(out_file, four_cc, 20, (x_shape, y_shape))
while True:
    start_time = time()
    ret, frame = player.read()
    assert ret
    results = score_frame(frame)
    frame = plot_boxes(results, frame)
    end_time = time()
    fps = 1/np.round(end_time - start_time, 3)
    #print(f"Frames Per Second : {fps}")
    #out.write(frame)